# Task 2: LDA Topic Model
***

We create a simple, out-of-the-box Latent Dirichlet Allocation Topic Model with visualization.

In [15]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import pandas as pd
import numpy as np

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

from sklearn.decomposition import LatentDirichletAllocation

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [16]:
df = pd.read_csv('cleaned_jels.csv', encoding='utf-8-sig', low_memory=False)
df.drop('0', axis=1, inplace=True)
df

,Title,Publisher,Abstract,Keywords,Jel Codes,Author1,Author2,Author3,Author4,Author5,...,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80
0,technical efficiency of hungarian farms before...,european association of agricultural economists,hungary is one of the ten countries that have ...,"[agribusiness, production economics]",NaN,"zoltán bakucs, lajos","fertő, imre","fogarasi, józsef",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,exploring the flexibility of polish family far...,european association of agricultural economists,no abstract is available for this item,"[farm management, productivity analysis]",NaN,"pieniadz, agata","renner, swetlana","petrick, martin",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,regional clusters in a function of rural devel...,european association of agricultural economists,as a theoretical concept rural development bas...,[community rural urban development],NaN,"sudarić, tihana","zmaić, krunoslav","petrač, božidar",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,the extended metropolitan area in a new member...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"zolin, m. bruna",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,infrastructural capacity of family owned holdi...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, land econo...",NaN,"Živković, dragić","dimitrijević, bojan","jelić, sreten","rajić, zoran",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,comparative research of food consumption in se...,european association of agricultural economists,no abstract is available for this item,[food consumption nutrition food safety],NaN,"Мilanović, milan r.","Đorović, milutin","stevanović, simo",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,targeting agricultural and rural development m...,european association of agricultural economists,no abstract is available for this item,"[agricultural and food policy, community rural...",NaN,"segré, andrea","rakić, renata","rokvić, gordana","vittuari, matteo",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,needs assessment analysis of small rural house...,european association of agricultural economists,no abstract is available for this item,[consumer household economics],NaN,"bogdanov, natalija","moslavac, nenad",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,designing a rural development strategy for ser...,european association of agricultural economists,no abstract is available for this item,[community rural urban development],NaN,"cochrane, nancy","reed, michael","jovanović, zlatko",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,rural development and the heritage of chayanov...,european association of agricultural economists,no abstract is available for this item,"[community rural urban development, labor and ...",NaN,"nuppenau, ernst-august",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


>We need to lemmatize all words to reduce dimensionality: jogging, jogged, jogs -> jog

In [17]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_tag(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    elif pos_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return None


class Lemmatizer_CountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(Lemmatizer_CountVectorizer, self).build_analyzer()
        return lambda doc:([lemmatizer.lemmatize(word[0], get_wordnet_tag(word[1])) for word in pos_tag(analyzer(doc)) 
                            if get_wordnet_tag(word[1]) is not None])

In [18]:
count_vectorizer = Lemmatizer_CountVectorizer(stop_words='english', lowercase=True, max_df=.5, min_df=10)
dtm_cv = count_vectorizer.fit_transform(df['Abstract'])

>We use a Latent Dirichlet Allocation Model. 

In [19]:
lda_cv = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_cv.fit(dtm_cv)

/usr/local/lib/python3.5/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7, learning_method=None,
             learning_offset=10.0, max_doc_update_iter=100, max_iter=10,
             mean_change_tol=0.001, n_jobs=1, n_topics=20, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [21]:
pyLDAvis.sklearn.prepare(lda_cv, dtm_cv, count_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      16.264931        1       1 -0.167818 -0.001867
8      14.202389        1       2 -0.190380  0.083626
15      6.742651        1       3 -0.108309 -0.017543
16      6.289385        1       4 -0.102489  0.103156
11      6.270321        1       5 -0.144703  0.016676
9       5.945973        1       6 -0.137432 -0.102243
1       5.376400        1       7  0.307274  0.056293
4       4.974992        1       8 -0.032707 -0.245395
7       4.110379        1       9 -0.086382 -0.066664
2       3.819832        1      10 -0.064145  0.176552
17      3.670863        1      11 -0.098021  0.179543
6       3.649419        1      12 -0.057827 -0.114817
12      3.538144        1      13  0.012196 -0.119297
10      3.376646        1      14 -0.093802  0.075763
14      3.292265        1      15 -0.013501 -0.178256
13      2.155953        1      16  0.072246  0.202380
19      1.819425        1      17  0.305562  0.041345
3       1.539148        1      18  0.182537 -0.114597
0       1.491350        1      19  0.194956  0.071372
18      1.469535        1      20  0.222745 -0.046026, topic_info=      Category          Freq          Term         Total  loglift  logprob
term                                                                      
13176  Default  36322.000000         price  36322.000000  30.0000  30.0000
10834  Default  55541.000000         model  55541.000000  29.0000  29.0000
3690   Default  32386.000000       country  32386.000000  28.0000  28.0000
71     Default  23859.000000      abstract  23859.000000  27.0000  27.0000
9234   Default  23751.000000          item  23751.000000  26.0000  26.0000
10297  Default  42314.000000        market  42314.000000  25.0000  25.0000
1350   Default  25244.000000     available  25244.000000  24.0000  24.0000
6704   Default  22502.000000          firm  22502.000000  23.0000  23.0000
6467   Default  16701.000000          farm  16701.000000  22.0000  22.0000
9543   Default  18280.000000            la  18280.000000  21.0000  21.0000
17054  Default  18248.000000         trade  18248.000000  20.0000  20.0000
13841  Default  24971.000000          rate  24971.000000  19.0000  19.0000
15989  Default  16593.000000         state  16593.000000  18.0000  18.0000
1470   Default  16200.000000          bank  16200.000000  17.0000  17.0000
3995   Default  28455.000000          data  28455.000000  16.0000  16.0000
17786  Default  54864.000000           use  54864.000000  15.0000  15.0000
5981   Default  21308.000000      estimate  21308.000000  14.0000  14.0000
14721  Default  16815.000000          risk  16815.000000  13.0000  13.0000
6838   Default  13823.000000          food  13823.000000  12.0000  12.0000
6666   Default  16233.000000     financial  16233.000000  11.0000  11.0000
17872  Default  15057.000000         value  15057.000000  10.0000  10.0000
3416   Default  12021.000000      consumer  12021.000000   9.0000   9.0000
426    Default  12652.000000  agricultural  12652.000000   8.0000   8.0000
8684   Default  15291.000000   information  15291.000000   7.0000   7.0000
3654   Default  24227.000000          cost  24227.000000   6.0000   6.0000
8469   Default  16892.000000        income  16892.000000   5.0000   5.0000
15396  Default   8833.000000         shock   8833.000000   4.0000   4.0000
7601   Default  17422.000000        growth  17422.000000   3.0000   3.0000
6469   Default  10275.000000        farmer  10275.000000   2.0000   2.0000
12810  Default  37662.000000        policy  37662.000000   1.0000   1.0000
...        ...           ...           ...           ...      ...      ...
2886   Topic20    979.400259      colombia    980.393888   4.2192  -4.6210
10945  Topic20    850.606895         moral    851.600523   4.2191  -4.7620
10313  Topic20    755.668389      marriage    756.662018   4.2189  -4.8803
2278   Topic20    693.293513           car    694.287142   4.2188  -4